In [14]:
import csv
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, SKOS, XSD

def make_ontology(csv_file, base_url):
    # New graph
    g = Graph()

    # Set up namespaces
    units = Namespace("https://vocab.sentier.dev/units/")
    qudt = Namespace("http://qudt.org/schema/qudt/")
    mrf = Namespace(base_url)
    g.bind("units", units)
    g.bind("qudt", qudt)
    g.bind("mrf", mrf)
    g.bind("skos", SKOS)

    # the existing Efficiency concept in chris's thing
    efficiency_qk = URIRef("https://vocab.sentier.dev/units/quantity-kind/Efficiency")

    # make the concept thingy
    scheme = URIRef(base_url)
    g.add((scheme, RDF.type, SKOS.ConceptScheme))
    g.add((scheme, SKOS.prefLabel, Literal("MRF Equipment Efficiency Units", lang="en")))

    # Grab CSV data
    with open(csv_file, 'r') as file:
        csv_stuff = csv.reader(file)
        headers = next(csv_stuff)  # Ditch first column (nevermind) 
        next(csv_stuff)  # Skip efficiency values row

    # Loop through and make concepts
    for header in headers:
        # Make URI
        uri = URIRef(f"{base_url}unit/{header.replace(' ', '_')}")

        # Add stuff to graph
        g.add((uri, RDF.type, SKOS.Concept))
        g.add((uri, SKOS.prefLabel, Literal(f"Efficiency of {header.replace('_', ' ')}", lang="en")))
        g.add((uri, SKOS.inScheme, scheme))
        g.add((uri, qudt.hasQuantityKind, efficiency_qk))
        g.add((uri, SKOS.broader, efficiency_qk))
        g.add((uri, SKOS.notation, Literal(f"{header.replace(' ', '_')}_eff", datatype=qudt.symbol)))
        g.add((uri, SKOS.definition, Literal(f"Ratio of output to input for {header.replace('_', ' ')} in Material Recovery Facility", lang="en")))
        g.add((uri, qudt.latexSymbol, Literal(f"\\eta_{{{header.replace(' ', '_')}}}", datatype=qudt.LatexString)))
        g.add((uri, qudt.informativeReference, Literal("https://en.wikipedia.org/wiki/Material_recovery_facility", datatype=XSD.anyURI)))

    return g

# Do the thing
csv_file = "corrected_mrf_equipment_efficiency.csv"
base_url = "https://vocab.sentier.dev/model-terms/mrf-equipment/"

ontology = make_ontology(csv_file, base_url)

# Save it
output_file = "mrf_equipment_efficiency_ontology.ttl"
ontology.serialize(destination=output_file, format="turtle")

print(f"Done! Saved to {output_file}")

Done! Saved to mrf_equipment_efficiency_ontology.ttl
